<a href="https://colab.research.google.com/github/atharvakale31/Early-prediction-of-Alzheimer-using-acoustic-biomarkers/blob/main/Voice_Recording_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai-whisper
!pip install pandas
!pip install pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=e074d23c7870d3139d99439e0176c50125f3b645cad9394f588a223b05171084
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [5]:
import zipfile
import os
import pandas as pd
import whisper
import warnings
from pydub import AudioSegment
from tqdm import tqdm

# Ignore warnings
warnings.filterwarnings("ignore")

# Set up paths
zip_path = '/content/test_audios.zip'  # Replace with your ZIP file path
output_dir = '/content/audio_files'     # Directory to extract files
csv_output = '/content/transcriptions_test_audios.csv'

# Load Whisper model (You can choose a specific model like 'base', 'small', etc.)
model = whisper.load_model("medium")

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

# Prepare CSV and batch processing variables
transcriptions = []
batch_size = 300

# Function to process audio file
def transcribe_audio(file_path, file_name):
    # Load and preprocess audio
    audio = AudioSegment.from_file(file_path)
    audio_length = audio.duration_seconds

    # Convert audio to a format compatible with Whisper if needed
    if audio.channels > 1:
        audio = audio.set_channels(1)
    audio.export("temp.wav", format="wav")

    # Transcribe using Whisper
    transcription = model.transcribe("temp.wav")
    text = transcription["text"].strip()

    return file_name, audio_length, text

# Process files and save results in batches
audio_files = os.listdir(output_dir)
for idx, file_name in enumerate(tqdm(audio_files, desc="Processing files")):
    file_path = os.path.join(output_dir, file_name)

    # Check if file is MP3
    if not file_name.lower().endswith(".mp3"):
        print(f"Skipping non-MP3 file: {file_name}")
        continue

    try:
        # Transcribe and save result
        result = transcribe_audio(file_path, file_name)
        transcriptions.append(result)

        # Save results in batches
        if (idx + 1) % batch_size == 0:
            pd.DataFrame(transcriptions, columns=["File Name", "Audio Length", "Transcription"]).to_csv(
                csv_output, mode='a', header=not os.path.exists(csv_output), index=False)
            transcriptions = []  # Reset the batch list

    except Exception as e:
        print(f"Error processing {file_name}: {e}")
        continue

# Save any remaining results after the loop
if transcriptions:
    pd.DataFrame(transcriptions, columns=["File Name", "Audio Length", "Transcription"]).to_csv(
        csv_output, mode='a', header=not os.path.exists(csv_output), index=False)

print("Transcription process completed!")


Processing files: 100%|██████████| 412/412 [30:16<00:00,  4.41s/it]

Transcription process completed!
